## Import Libraries

In [1]:
import os
user_dev_path = os.path.dirname(os.getcwd())
# get parent path for package
package_path = os.path.dirname(user_dev_path)
import sys
import pandas as pd
import yaml
import datetime as dt
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import glob
sys.path.insert(0,package_path + '/src/')
from hiv_support_package import visit_features,helper_functions
from sklearn.impute import SimpleImputer
from datetime import timedelta, date
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Loading data

In [2]:
try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [3]:
# Paths
input_root = config['rise_files']['raw_data_path']
# prepared datasets
otz_file = config['rise_files']['raw_otz_file']
eac_file = config['rise_files']['raw_eac_file']
otz_input = input_root + otz_file
eac_input = input_root + eac_file
print(otz_input)
print(eac_input)

/data/rise_data/OtzData.xlsx
/data/rise_data/EacData.xlsx


In [4]:
def load_excel_sheet(path: str):
    if path is not None:
        try:
            xls = pd.ExcelFile(path)
            df = xls.parse(skiprows=0)
            return df
        except Exception as e:
            print(f"Exception on loading excel spreadsheet with error: {e}")

In [5]:
df_otz = load_excel_sheet(otz_input)
df_eac = load_excel_sheet(eac_input)

### Standardize Columns

In [6]:
# Uppercase the column names
df_otz.columns= df_otz.columns.str.upper()
df_eac.columns = df_eac.columns.str.upper()

In [7]:
# Convert to datetime
df_eac['DATE_EAC1'] = pd.to_datetime(df_eac['DATE_EAC1'],errors="coerce")
df_eac['DATE_EAC2'] = pd.to_datetime(df_eac['DATE_EAC2'],errors="coerce")
df_eac['DATE_EAC3'] = pd.to_datetime(df_eac['DATE_EAC3'],errors="coerce")
df_otz['DATE_ENROLLED_OTZ'] = pd.to_datetime(df_otz['DATE_ENROLLED_OTZ'],errors="coerce")
df_otz['DATE_ENROLLED_OTZ_PLUS'] = pd.to_datetime(df_otz['DATE_ENROLLED_OTZ_PLUS'],errors="coerce")
df_otz['DATE_MODULE1'] = pd.to_datetime(df_otz['DATE_MODULE1'],errors="coerce")
df_otz['DATE_MODULE2'] = pd.to_datetime(df_otz['DATE_MODULE2'],errors="coerce")
df_otz['DATE_MODULE3'] = pd.to_datetime(df_otz['DATE_MODULE3'],errors="coerce")
df_otz['DATE_MODULE4'] = pd.to_datetime(df_otz['DATE_MODULE4'],errors="coerce")
df_otz['DATE_MODULE5'] = pd.to_datetime(df_otz['DATE_MODULE5'],errors="coerce")
df_otz['DATE_MODULE6'] = pd.to_datetime(df_otz['DATE_MODULE6'],errors="coerce")
df_otz['DATE_MODULE7'] = pd.to_datetime(df_otz['DATE_MODULE7'],errors="coerce")
df_otz['DATE_TRANSITIONED'] = pd.to_datetime(df_otz['DATE_TRANSITIONED'],errors="coerce")

In [8]:
df_eac = df_eac.sort_values(by=['PATIENT_ID','DATE_EAC1'])
df_otz = df_otz.sort_values(by=['PATIENT_ID','DATE_ENROLLED_OTZ'])

In [9]:
df_eac.drop(columns=['DATE_SAMPLE_COLLECTED','NOTES','DATE_LAST_VIRAL_LOAD'],inplace=True)
df_otz.drop(columns=['DATE_ENROLLED_OTZ_PLUS','DATE_TRANSITIONED'],inplace=True)

### De-Duplicating Dataframes

In [10]:
df_eac = df_eac.drop_duplicates()
df_otz = df_otz.drop_duplicates()

In [11]:
df_eac = df_eac[(df_eac.groupby('PATIENT_ID')
      ['DATE_EAC1']
      .diff(-1).dt.days.mul(-1) # Calculate the difference
      .fillna(float('inf'))     # make sure last row is kept
      .ge(5)                    # select diff >= 5
   )]

In [12]:
df_otz = df_otz[(df_otz.groupby('PATIENT_ID')
      ['DATE_ENROLLED_OTZ']
      .diff(-1).dt.days.mul(-1) # calculate the difference
      .fillna(float('inf'))     # make sure last row is kept
      .ge(2)                    # select diff >= 2
   )]

In [13]:
df_eac_copy = df_eac.copy()
df_eac_copy.drop(columns=['STATE','L.G.A','FACILITY NAME','EAC_ID','FACILITY_ID','LAST_VIRAL_LOAD'],inplace=True)
df_eac_pivot = (
    df_eac_copy.set_index('PATIENT_ID')
        .stack(dropna=True)
        .droplevel(1)
        .reset_index(name='DATE')
)

In [14]:
df_otz_copy = df_otz.copy()
df_otz_copy.drop(columns=['STATE','L.G.A','FACILITY NAME','OTZ_ID','FACILITY_ID','FULL_DISCLOSURE'],inplace=True)
df_otz_pivot = (
    df_otz_copy.set_index('PATIENT_ID')
        .stack(dropna=True)
        .droplevel(1)
        .reset_index(name='DATE')
)

## Feature Creation

In [15]:
# Count Number of appointments
df_eac_pivot['EAC_COUNT'] = df_eac_pivot.groupby('PATIENT_ID')["DATE"].rank("dense").astype(int)
df_otz_pivot['OTZ_COUNT'] = df_otz_pivot.groupby('PATIENT_ID')["DATE"].rank("dense").astype(int)

In [16]:
# Time in Enhanced Adherence Counselling
df_eac_pivot['EF_FIRST_DATE_EAC'] = df_eac_pivot.groupby('PATIENT_ID')['DATE'].transform('min')
df_eac_pivot['EF_MONTHS_SINCE_FIRST_EAC'] = ((df_eac_pivot['DATE'] - df_eac_pivot['EF_FIRST_DATE_EAC'])/np.timedelta64(1, 'M'))
df_eac_pivot['EF_MONTHS_SINCE_FIRST_EAC'] = df_eac_pivot['EF_MONTHS_SINCE_FIRST_EAC'].fillna(0).astype(int)

In [17]:
# Time in OTZ
df_otz_pivot['EF_FIRST_DATE_OTZ'] = df_otz_pivot.groupby('PATIENT_ID')['DATE'].transform('min')
df_otz_pivot['EF_DAYS_SINCE_FIRST_OTZ'] = ((df_otz_pivot['DATE'] - df_otz_pivot['EF_FIRST_DATE_OTZ'])/np.timedelta64(1, 'D'))
df_otz_pivot['EF_DAYS_SINCE_FIRST_OTZ'] = df_otz_pivot['EF_DAYS_SINCE_FIRST_OTZ'].fillna(0).astype(int)

## Push to VM

In [21]:
df_eac_pivot = df_eac_pivot.add_prefix('E_')
df_otz_pivot = df_otz_pivot.add_prefix('O_')

In [22]:
df_eac_pivot.to_csv(input_root+'feature_eac.csv',index=False,header=True)
df_otz_pivot.to_csv(input_root+'feature_otz.csv',index=False,header=True)